In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from numpy import linalg as LA

In [2]:

# Users
U = list(range(0, 8))

# Servers
S = list(range(0, 3))

# R = [CPU, RAM, VRAM, bandwidth]
R = np.zeros((4), dtype=int)

# Criticality

# Users u has memory requirements = w_j
#w = [[user_CPU for j in U],[user_RAM for j in U],[user_storage for j in U],[user_BW for j in U]]
#w = np.full((len(D), len(U)), 2)

# Each server has memory capacity C
#C= [[server_CPU for i in S],[server_RAM for i in S],[server_storage for i in S],[server_BW for i in S]]
#C = np.full((len(D), len(S)), 10)

# User u has memory requirements = w
W = []
NORM = []
for j in U:
  w = np.random.randint(1,4,4) # 4 random integers from the range of 1 to 3.
  W.append(w)
  norm = LA.norm(w)
  NORM.append(norm)

index = sorted(range(len(NORM)), key=lambda a: NORM[a])
print(index)

# Each server has memory capacity c
#C = np.full((len(D), len(S)), 10)
C = []
for i in S:
  c = np.random.randint(8,11,4) # 4 random integers from the range of 8 to 10.
  C.append(c)
 



[4, 2, 0, 1, 3, 7, 5, 6]


In [3]:
"""
# Users
U = list(range(0, 8))

# Servers
S = list(range(0, 3))

# D = [CPU, RAM, storage, bandwidth]
D = np.zeros((4), dtype=int)

w1 = [2, 3, 1, 1]
w2 = [3, 3,	2, 1]
w3 = [1, 3,	1, 3]
w4 = [2, 3,	3, 1]
w5 = [3, 2,	2, 3]
w6 = [3, 1,	1, 2]
w7 = [2, 2, 3, 1]
w8 = [2, 3,	1, 3]
W = [w1, w2, w3, w4, w5, w6, w7, w8]


c1 = [10, 10, 8, 9]
c2 = [8, 10, 8, 10]
c3 = [10, 10, 9, 8]
C = [c1, c2, c3]


NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8)]

index = sorted(range(len(NORM)), key=lambda a: NORM[a])
"""

'\n# Users\nU = list(range(0, 8))\n\n# Servers\nS = list(range(0, 3))\n\n# D = [CPU, RAM, storage, bandwidth]\nD = np.zeros((4), dtype=int)\n\nw1 = [2, 3, 1, 1]\nw2 = [3, 3,\t2, 1]\nw3 = [1, 3,\t1, 3]\nw4 = [2, 3,\t3, 1]\nw5 = [3, 2,\t2, 3]\nw6 = [3, 1,\t1, 2]\nw7 = [2, 2, 3, 1]\nw8 = [2, 3,\t1, 3]\nW = [w1, w2, w3, w4, w5, w6, w7, w8]\n\n\nc1 = [10, 10, 8, 9]\nc2 = [8, 10, 8, 10]\nc3 = [10, 10, 9, 8]\nC = [c1, c2, c3]\n\n\nNORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8)]\n\nindex = sorted(range(len(NORM)), key=lambda a: NORM[a])\n'

In [4]:
########## DECLARE THE MIP Solver ##########
solver = pywraplp.Solver.CreateSolver('SCIP')

In [5]:
########## DEFINE THE VARIABLES ##########
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in index:
    for i in S:
        x[i, j] = solver.BoolVar(f'c[{i}][{j}]')
        #print(x[i,j])

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in S:
    y[i] = solver.BoolVar(f'c[{i}]')

print('Number of variables =', solver.NumVariables())

Number of variables = 27


In [6]:
########## DEFINE THE CONSTRAINTS ##########

#Proximity constraint
#for j in U:
    #for i in S:
        # Missing coverage condition: 
        #solver.Add(
            #x[i, j]==0
        #)

# For each server i
# the sum of memory requirements of the users allocated to server i
# does not exceed the memory capacity of i
for i in range(len(S)):
    for k in range(len(D)):
        solver.Add(
            solver.Sum([
                x[i, j] * W[j][k] for j in index
                #x[i, j] * W[j][k] for j in range(len(U))
            ]) <= C[i][k]*y[i]
        )

# Each user can be allocated just once
for j in index:
    solver.Add(
        solver.Sum([x[i, j] for i in S]) <= 1
    )

# Constraint 3: x, y ≥ 0
# has already been included in the definition of the variables

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 20


In [7]:
########## DEFINE THE OBJECTIVE ##########

# Maximize the number of allocated users
objective = solver.Objective()
for j in index:
    for i in S:
        objective.SetCoefficient(x[i, j], 1)
objective.SetMaximization()

# Minimize the number of required edge servers
#objectiveMin = solver.Objective()

#for i in S:
 #   objectiveMin.SetCoefficient(y[i], 1)
#objectiveMin.SetMinimization()

In [8]:
########## CALL THE SOLVER ##########
status = solver.Solve()


########### DISPLAY THE SOLUTION ##########
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for j in index:
        for i in S:
            if int(x[i,j].solution_value()) == 1:
                print(f'User {j} has been allocated on server {i}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 8.0
User 4 has been allocated on server 0
User 2 has been allocated on server 0
User 0 has been allocated on server 0
User 1 has been allocated on server 0
User 3 has been allocated on server 1
User 7 has been allocated on server 1
User 5 has been allocated on server 1
User 6 has been allocated on server 2
